In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
query = """
select * from tables a
join tables_2 b
on b.call=a.del
"""

In [3]:
import re

def extract_sql_metadata(query):
    # Preprocessing steps to clean the query
    query = re.sub(r'--.*', '', query)  # Remove line comments
    query = re.sub(r'/\*.*?\*/', '', query, flags=re.DOTALL)  # Remove block comments
    query = re.sub(r"'\w+'", '', query)  # Remove string literals
    query = re.sub(r'\bEXTRACT\s*\([^)]+\)', '', query, flags=re.IGNORECASE)
    
    # Extract CTE names to exclude them later
    cte_pattern = r"with\s+([a-zA-Z0-9_]+)\s+as\s*\("
    ctes = set(re.findall(cte_pattern, query, re.IGNORECASE))
    
    # Identify physical tables (schema.table or single-part) from FROM/JOIN clauses
    table_pattern = r"\b(?:FROM|JOIN)\s+([a-zA-Z0-9_.]+)\b"
    candidates = re.findall(table_pattern, query, re.IGNORECASE)
    tables = sorted({
        c.lower()
        for c in candidates
        if c.lower() not in ctes  # Removed the '.' check
    })
    
    # Map table aliases to their physical tables
    alias_pattern = r'(?i)(?:FROM|JOIN)\s+([a-zA-Z0-9_.]+)(?:\s+(?:AS\s+)?([a-zA-Z0-9_]+))?'
    alias_map = {}
    for table, alias in re.findall(alias_pattern, query):
        table_lower = table.lower()
        if table_lower in tables:
            alias_lower = (alias or table.split('.')[-1]).lower()
            alias_map[alias_lower] = table_lower
    
    # Extract columns while ignoring aliases
    column_pattern = r'\b([a-zA-Z0-9_]+)\.([a-zA-Z0-9_]+)\b'
    column_dict = {t: set() for t in tables}
    
    for prefix, col in re.findall(column_pattern, query, re.IGNORECASE):
        table = alias_map.get(prefix.lower())
        if table and table in column_dict:
            column_dict[table].add(col.lower())
    
    # Convert sets to sorted lists
    for table in column_dict:
        column_dict[table] = sorted(column_dict[table])
    
    return {'tables': tables, 'columns': column_dict}

In [9]:
import re
import pandas as pd

def extract_sql_metadata(query):
    # Preprocessing steps to clean the query
    query = re.sub(r'--.*', '', query)  # Remove line comments
    query = re.sub(r'/\*.*?\*/', '', query, flags=re.DOTALL)  # Remove block comments
    query = re.sub(r"'\w+'", '', query)  # Remove string literals
    query = re.sub(r'\bEXTRACT\s*\([^)]+\)', '', query, flags=re.IGNORECASE)
    
    # Extract CTE names and their columns
    cte_pattern = r"WITH\s+([a-zA-Z0-9_]+)\s+AS\s*\(([^)]+)\)"
    ctes = re.findall(cte_pattern, query, re.IGNORECASE)
    cte_columns = {}
    for cte_name, cte_query in ctes:
        # Extract columns from the CTE query
        columns = re.findall(r'\bSELECT\s+([a-zA-Z0-9_,\s]+)\b', cte_query, re.IGNORECASE)
        if columns:
            # Split columns by comma and clean them
            cte_columns[cte_name.lower()] = [col.strip().lower() for col in columns[0].split(',')]
    
    # Identify physical tables (schema.table or single-part) from FROM/JOIN clauses
    table_pattern = r"\b(?:FROM|JOIN)\s+([a-zA-Z0-9_.]+)\b"
    candidates = re.findall(table_pattern, query, re.IGNORECASE)
    tables = sorted({
        c.lower()
        for c in candidates
        if c.lower() not in cte_columns  # Exclude CTEs from physical tables
    })
    
    # Map table aliases to their physical tables
    alias_pattern = r'(?i)(?:FROM|JOIN)\s+([a-zA-Z0-9_.]+)(?:\s+(?:AS\s+)?([a-zA-Z0-9_]+))?'
    alias_map = {}
    for table, alias in re.findall(alias_pattern, query):
        table_lower = table.lower()
        if table_lower in tables:
            alias_lower = (alias or table.split('.')[-1]).lower()
            alias_map[alias_lower] = table_lower
    
    # Extract columns while ignoring aliases
    column_pattern = r'\b([a-zA-Z0-9_]+)\.([a-zA-Z0-9_]+)\b'
    column_dict = {t: set() for t in tables}
    
    for prefix, col in re.findall(column_pattern, query, re.IGNORECASE):
        table = alias_map.get(prefix.lower())
        if table and table in column_dict:
            column_dict[table].add(col.lower())
    
    # Add CTE columns to the column_dict
    for cte_name, columns in cte_columns.items():
        column_dict[cte_name] = set(columns)
    
    # Convert sets to sorted lists
    for table in column_dict:
        column_dict[table] = sorted(column_dict[table])
    
    # Flatten the column_dict into a list of tuples
    table_column_pairs = []
    for table, columns in column_dict.items():
        for column in columns:
            table_column_pairs.append((table, column))
    
    # Create a DataFrame
    df = pd.DataFrame(table_column_pairs, columns=['Table', 'Column'])
    
    return df

# Example usage
query = """
WITH cte AS (
    SELECT id, name, col_x FROM public.users
)
SELECT u.id, u.name, o.order_id
FROM public.users u
JOIN orders o ON u.id = o.user_id
WHERE u.name = 'John'
JOIN cte ON cte.id = u.id
AND cte.col_x IS NOT NULL
"""

result_df = extract_sql_metadata(query)
print(result_df)

          Table             Column
0        orders           order_id
1        orders            user_id
2  public.users                 id
3  public.users               name
4           cte  col_x from public
5           cte                 id
6           cte               name
